<a href="https://colab.research.google.com/github/gurusaiedu/Gen-AI-hackathon-2024/blob/main/RAG_multi_model_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install Requirements
!apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow
# Install Python Packages
%pip install -q unstructured["local-inference"]==0.5.2 layoutparser
# upgrade to the latest, though has not been tested
# %pip install -q --upgrade unstructured layoutparser
%pip install -q "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"


Selecting previously unselected package poppler-utils.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Selecting previously unselected package tesseract-ocr-eng.
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr-osd.
Preparing to unpack .../tesseract-ocr-osd_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr.
Preparing to unpack .../tesseract-ocr_4.1.1-2.1build1_amd64.deb ...
Unpacking tesseract-ocr (4.1.1-2.1build1) ...
Setting up tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up tess

In [3]:
from unstructured.partition.pdf import partition_pdf

test_elements = partition_pdf("/content/Indian_Cushine.pdf")

model_final.pth:   0%|          | 0.00/330M [00:00<?, ?B/s]

(…)ayNet/faster_rcnn_R_50_FPN_3x/config.yml:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

In [4]:
type(test_elements)

list

In [5]:
len(test_elements)

414

In [6]:
test_elements[0]

In [8]:
from collections import Counter

display(Counter(type(element) for element in test_elements))

Counter({unstructured.documents.elements.NarrativeText: 325,
         unstructured.documents.elements.Title: 15,
         unstructured.documents.elements.FigureCaption: 69,
         unstructured.documents.elements.Text: 2,
         unstructured.documents.elements.ListItem: 3})

In [10]:
display(*[(type(element), element.text) for element in test_elements[10:15]])


(unstructured.documents.elements.NarrativeText,
 '3. **Shape Samosas**: Divide the dough into balls, roll them into ovals, cut in half, and form cones. Fill with potato mixture, seal edges.')

(unstructured.documents.elements.NarrativeText,
 'with potato mixture, seal edges.')

(unstructured.documents.elements.NarrativeText,
 '4. **Fry Samosas**: Deep fry in oil over medium heat until golden brown and crisp.')

(unstructured.documents.elements.FigureCaption, '')

(unstructured.documents.elements.NarrativeText,
 'A variety of Pakoras, including onion, potato, and spinach, coated in chickpea baDer and deep- fried to perfecXon, served with mint chutney and sliced raw onions.')

In [44]:
def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.FigureCaption" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.ListItem" in str(type(element)):
            texts.append(str(element))
    return texts, tables

texts, tables = categorize_elements(test_elements)


In [45]:
type(texts)

list

In [46]:
len(texts)

3

In [47]:
texts[0]

'Image of dal tadka as below:'

In [48]:
type(tables)

list

In [49]:
len(tables)

69

In [50]:
for i in tables:
  print(i)


Contents of 1 Samosa Quantity Calories 262 Fat 17 gm Carbohydrates 24 gm Fibre 2.1 gm Sugars 1.6 gm Protein 3.5 gm Cholesterol 27 mg Sodium 423 mg Potassium 189 mg Vitamins A (25%), C (17%) Calcium 2% Iron 4%

Contents of 100g Pakora Quantity Calories 287 Fat 11.1 gm Carbohydrates 37.3 gm Sugars 7.4 gm Protein 11.3 gm Sodium 17 mg Potassium 639 mg

Contents of 1 Aloo Tikki Quantity Calories 111 kcal Fat 4.23 gm Carbohydrates 18.15 gm Protein 2.38 gm

Contents of Paneer Tikka (1 piece) Quantity Calories 40 kcal Fat 1.46 gm Carbohydrates 3.1 gm Fibre 0.6gm Sugars 0.59 gm Protein 3.94 gm Cholesterol 4 mg Sodium 116 mg Potassium 80 mg Vitamins A (25%), C (17%) Calcium 2% Iron 4%


Contents of Butter Chicken (1 cup) Quantity Calories 2030 kcal 34.86 gm Carbohydrates 10.36 gm Fibre 2.9 gm Sugars 3.44 gm Protein 33.28 gm Cholesterol 126 mg Sodium 455 mg Potassium 537 mg

Contents of Palak Paneer (1 cup) Quantity Calories 405 kcal Fat 32.63gm Carbohydrates 14.56gm Fibre 3.6 gm Sugars 5.56 gm 

In [14]:
from unstructured.partition.pdf import partition_pdf

path = "/content/"
name = "Indian_Cushine.pdf"

elements=partition_pdf(
        filename=path + name,
        extract_images_in_pdf=False,
        infer_table_structure=True,
        chunking_strategy="by_title",
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
        # extract_image_block_output_dir=path,
    )


TypeError: partition_pdf() got an unexpected keyword argument 'extract_images_in_pdf'

In [12]:
help(partition_pdf)

Help on function partition_pdf in module unstructured.partition.pdf:

partition_pdf(filename: str = '', file: Optional[bytes] = None, url: Optional[str] = None, template: Optional[str] = None, token: Optional[str] = None, include_page_breaks: bool = False) -> List[unstructured.documents.elements.Element]
    Parses a pdf document into a list of interpreted elements.
    Parameters
    ----------
    filename
        A string defining the target filename path.
    file
        A file-like object as bytes --> open(filename, "rb").
    template
        A string defining the model to be used. Default None uses default model ("layout/pdf" url
        if using the API).
    url
        A string endpoint to self-host an inference API, if desired. If None, local inference will
        be used.
    token
        A string defining the authentication token for a self-host url, if applicable.



In [15]:
from unstructured.partition.pdf import partition_pdf

partition_pdf(
    filename=path + name,                  # mandatory
    strategy="hi_res",                                     # mandatory to use ``hi_res`` strategy
    extract_images_in_pdf=True,                            # mandatory to set as ``True``
    extract_image_block_types=["Image", "Table"],          # optional
    extract_image_block_to_payload=False,                  # optional
    extract_image_block_output_dir=path,  # optional - only works when ``extract_image_block_to_payload=False``
    )

TypeError: partition_pdf() got an unexpected keyword argument 'strategy'